In [1]:
import torch
import torch.nn as nn

class Discriminator(nn.Module):
    def __init__(self, channels_img, features_d):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            #Input: N x channels_img x 64 x 64
            nn.Conv2d(
                channels_img, features_d, kernel_size=4, stride=2, padding=1
            ),
            nn.LeakyReLU(0.2),
            self._block(features_d, features_d*2,4,2,1), #16x16
            self._block(features_d*2, features_d*4,4,2,1),#8x8
            self._block(features_d*4, features_d*8,4,2,1), #4x4
            nn.Conv2d(features_d*8, 1, kernel_size=4, stride=2, padding=0), #1x1
            nn.Sigmoid(),
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias = False
            ),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2), 
        )
        
    def forward(self, x):
        return self.disc(x)

In [2]:
class Generator(nn.Module):
    def __init__(self, z_dim, channels_img, features_g):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            self._block(z_dim,features_g*16, 4, 1, 0), # N x f_g*16 x 4 x 4
            self._block(features_g*16, features_g*8, 4, 2, 1), #8x8
            self._block(features_g*8, features_g*4, 4, 2, 1), #16x16
            self._block(features_g*4, features_g*2, 4, 2, 1), #32x32
            nn.ConvTranspose2d(
                features_g*2, channels_img, kernel_size=4, stride=2, padding=1
            ),
            nn.Tanh(), # [-1,1]
        )
    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(
                in_channels,
                out_channels, 
                kernel_size, 
                stride, 
                padding, 
                bias=False
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )
    def forward(self, x):
        return self.gen(x)

In [3]:
  def initialize_weights(model):
        for m in model.modules():
            if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
                nn.init.normal_(m.weight.data, 0.0, 0.02)          

In [4]:
 def test():
     N, in_channels, H, W = 8,3,64,64
     z_dim = 100
     x = torch.randn((N, in_channels, H, W))
     disc = Discriminator(in_channels, 8)
     initialize_weights(disc)
     assert disc(x).shape == (N,1,1,1)
     gen = Generator(z_dim, in_channels, 8)
     initialize_weights(gen)
     z = torch.randn((N, z_dim, 1, 1))
     assert gen(z).shape == (N, in_channels, H, W)
     print("Success")

In [5]:
test()

Success


In [6]:
# !ping -c 3 google.com
# %pip install --upgrade pip
# %pip install --upgrade torchvision
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

In [7]:
import torch.optim as optim
import torchvision
from torchvision import datasets as datasets
import torchvision.transforms as T
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

2025-05-05 01:05:08.300225: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746407108.629379      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746407108.707959      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


**Hyperparametros**

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LEARNING_RATE = 2e-4
BATCH_SIZE = 128
IMAGE_SIZE = 64
CHANNELS_IMG = 3
Z_DIM = 100
NUM_EPOCHS = 5
FEATURES_DISC = 64
FEATURES_GEN = 64

In [9]:
transforms = T.Compose(
    [
        T.Resize(IMAGE_SIZE),
        T.ToTensor(),
        T.Normalize([0.5 for _ in range(CHANNELS_IMG)], [0.5 for _ in range(CHANNELS_IMG)]),
    ]
)

In [10]:
# dataset = datasets.MNIST(
#     root="dataset/", 
#     train=True,
#     transform=transforms,
#     download=True,
# )

##Dataset MNSIT
# dataset = datasets.ImageFolder(
#     "/kaggle/input/mnsit-dataset",
#     transform=transforms
# )

dataset = datasets.ImageFolder(
    "/kaggle/input/celebahq256-images-only",
    transform=transforms
)

In [11]:
loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
gen = Generator(Z_DIM, CHANNELS_IMG, FEATURES_GEN).to(device)
disc = Discriminator(CHANNELS_IMG, FEATURES_DISC).to(device)
initialize_weights(gen)
initialize_weights(disc)

In [12]:
opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
opt_disc = optim.Adam(disc.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
criterion = nn.BCELoss()

fixed_noise = torch.randn(32, Z_DIM, 1,1).to(device)
writer_real = SummaryWriter(f"logs/real")
writer_fake = SummaryWriter(f"logs/fake")
step = 0

In [13]:
gen.train()
disc.train()

for epoch in range(NUM_EPOCHS):
    for batch_idx, (real, _) in enumerate(loader):
        real = real.to(device)
        noise = torch.randn((BATCH_SIZE,Z_DIM, 1,1)).to(device)
        fake = gen(noise)
    

        ### Train Discriminator max log(D(x)) + log(1- D(G(z)))
        disc_real = disc(real).reshape(-1)
        loss_disc_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake).reshape(-1)
        loss_disc_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        loss_disc = (loss_disc_real + loss_disc_fake ) / 2
        disc.zero_grad()
        loss_disc.backward(retain_graph=True)
        opt_disc.step()

        ### Train Discriminator min log(1 - D(G(z))) <--> max log(D(G(z)))
        output = disc(fake).reshape(-1)
        loss_gen = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()

        if batch_idx % 100 == 0:
            print(
                f"Epoch [{epoch}/{NUM_EPOCHS}] Batch {batch_idx}/{len(loader)} \ Loss D: {loss_disc:.4f}, loss G: {loss_gen:.4f}"
            )

            with torch.no_grad():
                fake = gen(fixed_noise)

                img_grid_real = torchvision.utils.make_grid(
                    real[:32], normalize=True
                )
                img_grid_fake = torchvision.utils.make_grid(
                    fake[:32], normalize=True
                )

                writer_real.add_image("Real", img_grid_real, global_step=step)
                writer_real.add_image("Fake", img_grid_fake, global_step=step)
                
            step += 1

Epoch [0/5] Batch 0/235 \ Loss D: 0.6880, loss G: 0.7970
Epoch [0/5] Batch 100/235 \ Loss D: 0.0257, loss G: 3.7894
Epoch [0/5] Batch 200/235 \ Loss D: 0.4776, loss G: 2.1897
Epoch [1/5] Batch 0/235 \ Loss D: 0.4126, loss G: 1.9187
Epoch [1/5] Batch 100/235 \ Loss D: 0.6031, loss G: 1.6967
Epoch [1/5] Batch 200/235 \ Loss D: 0.7021, loss G: 1.0030
Epoch [2/5] Batch 0/235 \ Loss D: 0.6511, loss G: 1.3044
Epoch [2/5] Batch 100/235 \ Loss D: 0.5300, loss G: 1.2238
Epoch [2/5] Batch 200/235 \ Loss D: 0.5726, loss G: 1.7945
Epoch [3/5] Batch 0/235 \ Loss D: 0.5837, loss G: 1.4620
Epoch [3/5] Batch 100/235 \ Loss D: 0.5430, loss G: 2.1590
Epoch [3/5] Batch 200/235 \ Loss D: 0.5367, loss G: 1.7734
Epoch [4/5] Batch 0/235 \ Loss D: 0.6181, loss G: 1.4330
Epoch [4/5] Batch 100/235 \ Loss D: 0.5801, loss G: 0.9887
Epoch [4/5] Batch 200/235 \ Loss D: 0.5882, loss G: 1.7871
